## MNS rental market reports

In this notebook, we will fetch data from downloaded [MNS PDFs](https://www.mns.com/bronx_rental_market_report).

In [1]:
# importing libraries

import pandas as pd 
import glob
import pymupdf4llm # works best for this case, but still needs clean-up
import re

## Monthly data

Previewing files, we know that what we need are the following:

* Page 4: shows average rent in the borough for the month
* Page 8: Mott Haven studio rental prices
* Page 9: Mott Haven one-bedroom rental prices
* Page 10: Mott Haven two-bedroom rental prices

First test showed us that the tables cannot be captured via `tabula`. Using `pymupdf4llm` and `mangoCR` requires clean-up.

In [2]:
# pulls files we need

monthly_pdfs = glob.glob("monthly-reports/*.pdf")
len(monthly_pdfs)

66

### Extracting monthly average rent in the Bronx

#### To make things easier for us, we will first define functions:

1. extracting PDF text via pymupdf4llm
2. extracting month/year and corresponding dollar value

In [3]:
# extracting PDF text via pymupdf4llm

def extract_text(pdf_path, page_num):
    """
    This function extracts text from a PDF page.
    It requires `pymupdf4llm` to run.

    Parameters:
    pdf_path (str): path to the PDF file
    page_num (int): the page number where text will be extracted -1, because this will run as a list
    """
    pdf_text = pymupdf4llm.to_markdown(pdf_path, pages=[page_num])
    return pdf_text

In [4]:
# extracting the average rent data per month

def extract_monthly_average(pdf_text):
    """
    This function extracts the most recent month/year and corresponding rent value from the PDF text.
    It requires `re` to run.
    
    Parameters:
    pdf_text (str): text extracted from the PDF
        
    Returns:
    month_year, rent_amount
    """
    try:
        # finding the most recent month and year via regex
        month_year_pattern = r"([a-zA-Z]+\s+\d{4})" # text characters + space or non-space + four-digit numbers
        month_year_matches = re.findall(month_year_pattern, pdf_text)
        if not month_year_matches:
            return None, None
        month_year = month_year_matches[-1] # gets the last match or most recent month/year
        
        # finding dollar amounts or corresponding rent value
        dollar_pattern = r"\$[\d+,]+" # dollar sign + digits
        dollar_matches = re.findall(dollar_pattern, pdf_text)
        if not dollar_matches:
            return None, None
        rent_amount = float(dollar_matches[-1].replace("$", "").replace(",", ""))
        
        return month_year, rent_amount
    
    except Exception as e:
        print(f"Error processing text: {e}")
        return None, None

#### Let's see magic happen!

Here's our code to extract average monthly rental prices in the Bronx.

In [5]:
counter = 0
bx_monthly_data = [] # holds our data

for pdf in monthly_pdfs:
    counter += 1
    print(f"Processing {counter} of {len(monthly_pdfs)}...")
    pdf_text = extract_text(pdf, 3)
    month_year, ave_monthly_rent = extract_monthly_average(pdf_text)
    bx_monthly_data.append({
        "filename": pdf,
        "period": month_year,
        "bx_average_rent": ave_monthly_rent,
    })
    if counter == len(monthly_pdfs):
        print("Done!")
    else:
        print("Done! Moving on to the next file...")

# saving into a df
bx_monthly_df = pd.DataFrame(bx_monthly_data)

Processing 1 of 66...
Processing monthly-reports/bronx_market_report_dec_20.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 2 of 66...
Processing monthly-reports/bronx_market_report_oct_22.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 3 of 66...
Processing monthly-reports/bronx_market_report_oct_23.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 4 of 66...
Processing monthly-reports/bronx_market_report_dec_21.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to t

### Extracting studio, one-bedroom, and two-bedroom average monthly rental prices in Mott Haven

We will be using the same `extract_text` function, but because the page format is different for the next pages we want to extract text from, we will be writing a new function.

In [6]:
# extracting the average monthly rent data per unit type in Mott Haven
# it looks like we don't have to make a lot of changes because Mott Haven is the last item per page

def extract_per_unit_average(pdf_text):
    """
    This function extracts the corresponding rent value from the PDF text.
    It requires `re` to run.
    
    Parameters:
    pdf_text (str): text extracted from the PDF
    """
    try:
        # finding dollar amounts or corresponding rent value
        dollar_pattern = r'\$[\d,]+' # dollar sign + digits
        dollar_matches = re.findall(dollar_pattern, pdf_text)
        if not dollar_matches:
            return None
        rent_amount = float(dollar_matches[-1].replace('$', '').replace(',', ''))
        
        return rent_amount
    
    except Exception as e:
        print(f"Error: {e}")
        return None

#### Let's see magic happen!

In [7]:
counter = 0
studio_monthly_data = [] # holds our data
one_bedroom_monthly_data = [] # holds our data
two_bedroom_monthly_data = [] # holds our data

for pdf in monthly_pdfs:
    counter += 1
    print(f"Processing {counter} of {len(monthly_pdfs)}...")
    
    # studio data is on page 8
    pdf_text = extract_text(pdf, 7)
    studio_monthly_rent = extract_per_unit_average(pdf_text)
    studio_monthly_data.append({
        "mott_haven_studio_rent": studio_monthly_rent,
        "filename": pdf
    })

    # one-bedroom data is on page 9
    pdf_text = extract_text(pdf, 8)
    one_bedroom_monthly_rent = extract_per_unit_average(pdf_text)
    one_bedroom_monthly_data.append({
        "mott_haven_one-bedroom_rent": one_bedroom_monthly_rent,
        "filename": pdf
    })

    # two-bedroom data is on page 10
    pdf_text = extract_text(pdf, 9)
    two_bedroom_monthly_rent = extract_per_unit_average(pdf_text)
    two_bedroom_monthly_data.append({
        "mott_haven_two-bedroom_rent": two_bedroom_monthly_rent,
        "filename": pdf
    })
    
    if counter == len(monthly_pdfs):
        print("Done!")
    else:
        print("Done! Moving on to the next file...")

# saving lists into dfs
studio_monthly_df = pd.DataFrame(studio_monthly_data)
one_bedroom_monthly_df = pd.DataFrame(one_bedroom_monthly_data)
two_bedroom_monthly_df = pd.DataFrame(two_bedroom_monthly_data)

Processing 1 of 66...
Processing monthly-reports/bronx_market_report_dec_20.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing monthly-reports/bronx_market_report_dec_20.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing monthly-reports/bronx_market_report_dec_20.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 2 of 66...
Processing monthly-reports/bronx_market_report_oct_22.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing monthly-reports/bronx_market_report_oct_22.pdf...
[                                        ] (0/1============================

#### Merging all the dfs

In [8]:
merged_monthly_df = pd.merge(bx_monthly_df, studio_monthly_df, on="filename", how="outer")
merged_monthly_df = pd.merge(merged_monthly_df, one_bedroom_monthly_df, on="filename", how="outer")
merged_monthly_df = pd.merge(merged_monthly_df, two_bedroom_monthly_df, on="filename", how="outer")

merged_monthly_df.to_csv("datasets/monthly_data.csv", encoding="UTF-8", index=False)

## Yearly data

Previewing files, we know that what we need are the following:
* Page 5: notable trends (text)
* Page 5: shows average rent in the borough for the year, across studio, one-bedroom, and two-bedroom units
* Page 6: Mott Haven mean studio rental prices
* Page 7: Mott Haven mean one-bedroom and two-bedroom rental prices

First test showed us that the tables cannot be captured via `tabula`. Using `pymupdf4llm` and `mangoCR` requires clean-up.

In [9]:
# pulls files we need

yearly_pdfs = glob.glob("yearend-reports/*.pdf")
len(yearly_pdfs)

4

### Extracting studio, one-bedroom, and two-bedroom average annual rental prices in the Bronx

... and the borough trends text for some context.

In [10]:
# extracting text in certain pages and lines

def get_detail(pdf_text, line_num):
    """
    This function fetches the text or detail through a specific line number from an extracted text.
    It requires `pymupdf4llm` to run. 

    Parameters:
    pdf_text (str): text extracted from PDF
    line_num (int): line number from the pdf_text that we will be using
    """
    try:
        target_detail = pdf_text.splitlines()[line_num]
        if "$" in target_detail:
            target_detail = float(target_detail.replace("$", "").replace(",", ""))
        return target_detail
    except Exception as e:
        print(f"Error: {e}")
        return None

In [11]:
counter = 0
bx_trends = [] # holds all trends texts
studio_annual_data = [] # holds our data
one_bedroom_annual_data = [] # holds our data
two_bedroom_annual_data = [] # holds our data

for pdf in yearly_pdfs:
    counter += 1
    print(f"Processing {counter} of {len(yearly_pdfs)}...")
    pdf_text = extract_text(pdf, 4)
    trends = get_detail(pdf_text, 1)
    bx_studio = get_detail(pdf_text, 9)
    bx_one_bedroom = get_detail(pdf_text, 11)
    bx_two_bedroom = get_detail(pdf_text, 13)

    bx_trends.append({
            "filename": pdf,
            "year": pdf[-8:-4],
            "trends": trends
    })

    studio_annual_data.append({
            "filename": pdf, # no need to add year because we will merge them later
            "bx_studio_rent": bx_studio
    })

    one_bedroom_annual_data.append({
            "filename": pdf, # no need to add year because we will merge them later
            "bx_one-bedroom_rent": bx_one_bedroom
    })

    two_bedroom_annual_data.append({
            "filename": pdf, # no need to add year because we will merge them later
            "bx_two-bedroom_rent": bx_two_bedroom
    })
    
    if counter == len(yearly_pdfs):
        print("Done!")
    else:
        print("Done! Moving on to the next file...")

# saving into a df
bx_trends_df = pd.DataFrame(bx_trends)
studio_annual_df = pd.DataFrame(studio_annual_data)
one_bedroom_annual_df = pd.DataFrame(one_bedroom_annual_data)
two_bedroom_annual_df = pd.DataFrame(two_bedroom_annual_data)

Processing 1 of 4...
Processing yearend-reports/bronx_year_end_market_report_2021.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 2 of 4...
Processing yearend-reports/bronx_year_end_market_report_2020.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 3 of 4...
Processing yearend-reports/bronx_year_end_market_report_2022.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Done! Moving on to the next file...
Processing 4 of 4...
Processing yearend-reports/bronx_year_end_market_report_2023.pdf...
[                                        ] (0/1=======================================[========================================] (1

#### Merging the dfs

In [12]:
merged_annual_df = pd.merge(studio_annual_df, one_bedroom_annual_df, on="filename", how="outer")
merged_annual_df = pd.merge(merged_annual_df, two_bedroom_annual_df, on="filename", how="outer")
merged_annual_df = pd.merge(merged_annual_df, bx_trends_df, on="filename", how="outer")

merged_annual_df.to_csv("datasets/annual_data.csv", encoding="UTF-8", index=False)